In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix,ConfusionMatrixDisplay

In [ ]:
#these are not exacly the same as the dataset 
class_names = ['air conditioner', 'car horn', 'children playing', 'dog bark', 'drilling',
               'engine idling', 'gun shot', 'jackhammer', 'siren', 'street music']

In [ ]:
def evaluate_model(model, X_val, y_val):
    y_pred = np.argmax(model.predict(X_val), axis=1)
    acc = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, average='macro')
    print("Accuracy:", acc)
    print("F1 Score (macro):", f1)
    print("\nClassification Report:\n", classification_report(y_val, y_pred))
    return acc, f1

In [ ]:
def plot_conf_matrix(model, X_test, y_test):
    y_pred = np.argmax(model.predict(X_test), axis=1)
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names) 
    disp.plot(cmap="Blues", xticks_rotation=45)
    plt.title("Confusion Matrix")
    plt.show()


In [ ]:
# ==== Default Hyperparameters ====
INPUT_SHAPE = (X_train.shape[1], X_train.shape[2])
NUM_CLASSES = 10
LSTM_UNITS = 64
DENSE_UNITS = 128
DROPOUT_RATE = 0.5
EPOCHS = 20
BATCH_SIZE = 32
LEARNING_RATE = 0.001
LOSS_FUNCTION = "sparse_categorical_crossentropy"
OPTIMIZER = "adam"

In [ ]:
def build_and_train_LSTM_model(model_name,input_shape=INPUT_SHAPE,lstm_units=LSTM_UNITS,dense_units=DENSE_UNITS,num_classes=NUM_CLASSES,epochs=EPOCHS ,dropout_rate=DROPOUT_RATE, learning_rate=LEARNING_RATE, batch_size=BATCH_SIZE,loss_function=LOSS_FUNCTION, optimizer=OPTIMIZER):
    
    model = Sequential()
    model.add(LSTM(lstm_units, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(lstm_units, return_sequences=False))
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))

    
    model.summary()

    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss=loss_function,
                  metrics=['accuracy'])

    
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    checkpoint = ModelCheckpoint(f'{model_name}.h5', monitor='val_loss', save_best_only=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)

    
    history = model.fit(X_train, y_train,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(X_val, y_val),
                        callbacks=[early_stopping, checkpoint, lr_scheduler])
    

    # Save Final Model
    #model.save(f'{model_name}.h5')
    
    
    plt.plot(history.history['loss'], label='Train Loss')    
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title('Loss Curve for {}'.format(model_name))
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    
    acc, f1 = evaluate_model(model, X_val, y_val)
    return model, history, acc, f1


    

In [ ]:
default = build_and_train_LSTM_model(model_name="default")


In [ ]:
lstm_32 =build_and_train_LSTM_model(model_name="lstm_32", lstm_units=32)


In [ ]:
lstm_128 = build_and_train_LSTM_model(model_name="lstm_128", lstm_units=128)


In [ ]:
dropout_03 =build_and_train_LSTM_model(model_name="dropout_0.3", dropout_rate=0.3)


In [ ]:
dropout_07 = build_and_train_LSTM_model(model_name="dropout_0.7", dropout_rate=0.7)


In [ ]:
lr_0005 =build_and_train_LSTM_model(model_name="lr_0005", learning_rate=0.0005)


In [ ]:
lr_005 = build_and_train_LSTM_model(model_name="lr_005", learning_rate=0.005)


In [ ]:
batch_64 = build_and_train_LSTM_model(model_name="batch_64", batch_size=64)


In [ ]:
evaluate_model(best_model, X_test, y_test)

In [ ]:
plot_conf_matrix(best_model, X_test, y_test, class_names)
